# How to use CrewAI Planning in your Crew of agents

---

The planning feature in CrewAI allows you to add planning capability to your crew. When enabled, before each Crew iteration, all Crew information is sent to an AgentPlanner that will plan the tasks step by step, and this plan will be added to each task description.

This guide will serve as a demostration of how to use the **planning** feature in your crews. Let's start by installing required packages.

### Install dependencies

In [ ]:
%pip install -U --quiet crewai crewai-tools exa_py
%pip install --quiet --upgrade ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.7/366.7 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.0/657.0 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta

In [ ]:
import warnings

# Filter out deprecation warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', message='Pydantic serializer warnings')

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### Configure API keys for both the LLM and Exa for searching the web.

In [2]:
import os
from crewai import LLM
from crewai_tools import EXASearchTool

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
EXA_API_KEY = os.getenv('EXA_API_KEY')

openai_model = LLM(model="openai/gpt-4o-mini", api_key=OPENAI_API_KEY)
tool = EXASearchTool(api_key=EXA_API_KEY)

/Users/anup/Desktop/Anup/crew_ai_project/.venv_manual/lib/python3.12/site-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


### Now we build our agents!
 
- **Analyst** to do research and a 
- **Reporter** to generate a detailed report from the research.
- In our **crew** we will add **`planning=True`** to enable planning capabilities for our agents before they perform the tasks. 
- Here's where we also add the **`planning_llm`** that will be used to plan the tasks

In [3]:
from crewai import Agent, Task, Crew

# Create an agent with reasoning enabled
analyst = Agent(
    role="Research Specialist",
    goal="Conduct detailed research on {topic} using the latest information as of {todays_date}.",
    backstory="You are an expert at web research.",
    llm=openai_model,
    tools=[tool],
    verbose=True
)

reporter = Agent(
    role="Report Specialist",
    goal="Generate a detailed report on {topic} using the latest information as of {todays_date}.",
    backstory="You are an expert at executive report writing",
    llm=openai_model,
    verbose=True
)

# Create tasks
analysis_task = Task(
    description="Research and analyze the recent trends and generate a detailed report on {topic}",
    expected_output="A detailed report structured as an executive report for c-suite execs to read.",
    agent=analyst,
    verbose=True
)

report_task = Task(
    description="Generate a detailed report on {topic}",
    expected_output="A detailed report structured as an executive report for c-suite execs to read.",
    agent=analyst,
    markdowns=True,
    verbose=True
)

# Create a crew and run the task
crew = Crew(
    agents=[analyst, reporter],
    tasks=[analysis_task, report_task],
    planning=True,
    planning_llm=openai_model,
    verbose=True
)

result = crew.kickoff(inputs={"topic":"Influencer marketing", "todays_date": "July 31, 2025"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 026c3f4d-f036-442a-a73e-1e8f29a90de7                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

 
[2025-08-24 12:44:11][INFO]: Planning the crew execution


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7196b617-e1a2-4ac9-8d7a-0f2d3915f549                                                                     │
│  Agent: Task Execution Planner                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Specialist                                                                                     │
│                                                                                                                 │
│  Task: Research and analyze the recent trends and generate a detailed report on Influencer marketing1. Begin    │
│  by clearly defining the 'search_query' that is relevant to recent trends in Influencer marketing. This could   │
│  include phrases like 'Influencer marketing trends 2025', 'effects of influencer marketing', 'top influencer    │
│  marketing strategies', etc.                                                                                    │
│  2. Set the 'start_published_date' to a suitable date that provides relevant recent data — for example,         │
│  '2024-01-01'. Set 'end_published_date' to '2025-07-31' to include all valid updates till July 31, 2025.        │
│  3. Prepare the necessary credentials and ensure that the EXA_API_KEY is valid and accessible in the            │
│  environment variables to use the EXASearchTool.                                                                │
│  4. Execute the EXASearchTool with the defined search_query, start_published_date, and end_published_date,      │
│  gathering relevant information on the latest trends in Influencer marketing. Carefully filter out results to   │
│  focus specifically on quality sources—look for published papers, industry reports, and expert opinions.        │
│  5. Gather insights from the search results, noting key statistics, quotes, and discoveries that relate to how  │
│  Influencer marketing has evolved, what platforms are most popular, and notable case studies.                   │
│  6. After compiling the information, analyze the data to identify patterns, changes, and new approaches to      │
│  influencer marketing that have emerged recently.                                                               │
│  7. Structure the findings in a coherent manner, making sure to summarize the key points first, followed by a   │
│  deeper analysis for each section, creating manageable subheadings for ease of reading.                         │
│  8. Conclude with an executive summary that provides high-level insights and actionable recommendations         │
│  suitable for c-suite executives. Ensure the tone is professional, direct, and insightful.                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Title: Creator Earnings Report 2025                                                                            │
│  URL: https://influencermarketinghub.com/creator-earnings-report-2025/                                          │
│  ID: https://influencermarketinghub.com/creator-earnings-report-2025/                                           │
│  Score: None                                                                                                    │
│  Published Date: 2025-08-04T00:00:00.000Z                                                                       │
│  Author: Nadica Naceva                                                                                          │
│  Image:                                                                                                         │
│  https://influencermarketinghub.com/wp-content/uploads/2025/08/20250804_1819_Creator-Earnings-Report-2025_remi  │
│  x_01k1txzmvte61r8vrzj698fcxs.jpg                                                                               │
│  Favicon: https://influencermarketinghub.com/favicon-32x32.png                                                  │
│  Extras: None                                                                                                   │
│  Subpages: None                                                                                                 │
│  Text: None                                                                                                     │
│  Highlights: None                                                                                               │
│  Highlight Scores: None                                                                                         │
│  Summary: None                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
│  Title: 5 influencer marketing predictions for the second half of 2025                                          │
│  URL: https://adage.com/trends-predictions/aa-5-influencer-marketing-predictions-second-half-2025/              │
│  ID: https://adage.com/trends-predictions/aa-5-influencer-marketing-predictions-second-half-2025/               │
│  Score: None                                                                                                    │
│  Published Date: 2025-08-06T00:00:00.000Z                                                                       │
│  Author: Gillian Follett                                                                                        │
│  Image: https://cloudfront-us-east-1.images.arcpublishing.com/crain/HYKGBRP7XZHOPLTJWUFCYWVVTU.jpg              │
│  Favicon: https://adage.com/pf/resources/images/favicons/adage.ico?d=139                                        │
│  Extras: None                                                                                                   │
│  Subpages: None                                                                                                 │
│  Text: None                                                                                                     │
│  Highlights: None                                                                                               │
│  Highlight Scores: None                                                                                         │
│  Summary: None                                                                                                  │
│                                                       

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Specialist                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Influencer Marketing Report 2025                                                                             │
│                                                                                                                 │
│  ## Executive Summary                                                                                           │
│  Influencer marketing in 2025 has evolved into a vital strategy for brands looking to enhance their reach and   │
│  engagement across digital platforms. Recent statistics indicate that influencer marketing spending in the      │
│  United States is projected to exceed $10 billion this year, demonstrating significant growth from previous     │
│  years. As consumers increasingly trust influencers over traditional advertising channels, businesses are       │
│  leveraging authentic connections fostered by these content creators. Emerging platforms like TikTok continue   │
│  to shape strategies, highlighting the need for brands to adapt to new trends quickly. This report outlines     │
│  the current market landscape, recent trends, successful case studies, and provides actionable recommendations  │
│  for C-suite executives to harness the full potential of influencer marketing.                                  │
│                                                                                                                 │
│  ## Market Overview                                                                                             │
│  The influencer marketing sector has seen substantial growth in recent years, resulting in a market value       │
│  estimated to reach approximately $16.4 billion in 2025. The annual growth rate for the industry is approx      │
│  29%, fueled by the increasing reliance on social media for marketing tactics. Studies show that brands         │
│  receive an average of $6.50 for every $1 spent on influencer marketing, showcasing superior ROI compared to    │
│  traditional marketing methods. As the landscape becomes increasingly competitive, companies are investing in   │
│  a diverse range of influencers, from micro to macro, to effectively tap into niche audiences.                  │
│                                                                                                                 │
│  ## Current Trends                                                                                              │
│  The following trends have emerged prominently in 2025:                                                         │
│  - **Increased Focus on Authenticity**: Brands are seeking partnerships with influencers who resonate with      │
│  their values and target audience, fostering trust and authenticity.                                            │
│  - **Rise of Short-Form Video Content**: Platforms like TikTok have solidified short-form videos as a           │
│  preferred format for engaging audiences, leading to innovative campaign strategies.                            │
│  - **Sustainability Claims**: The rise of eco-conscious consumers has propelled brands to collaborate with      │
│  influencers advocating for sustainability.                                                                     │
│  - **Diversity and Inclusion**: Campaigns now prominently feature diverse influencers to reflect broader        │
│  societal values and attract a wider audience.         

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3df36da4-3d6f-4490-baf2-995ef184b53a                                                                     │
│  Agent: Research Specialist                                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 026c3f4d-f036-442a-a73e-1e8f29a90de7                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Influencer Marketing Report 2025                                                               │
│                                                                                                                 │
│  ## Executive Summary                                                                                           │
│  Influencer marketing in 2025 has evolved into a vital strategy for brands looking to enhance their reach and   │
│  engagement across digital platforms. Recent statistics indicate that influencer marketing spending in the      │
│  United States is projected to exceed $10 billion this year, demonstrating significant growth from previous     │
│  years. As consumers increasingly trust influencers over traditional advertising channels, businesses are       │
│  leveraging authentic connections fostered by these content creators. Emerging platforms like TikTok continue   │
│  to shape strategies, highlighting the need for brands to adapt to new trends quickly. This report outlines     │
│  the current market landscape, recent trends, successful case studies, and provides actionable recommendations  │
│  for C-suite executives to harness the full potential of influencer marketing.                                  │
│                                                                                                                 │
│  ## Market Overview                                                                                             │
│  The influencer marketing sector has seen substantial growth in recent years, resulting in a market value       │
│  estimated to reach approximately $16.4 billion in 2025. The annual growth rate for the industry is approx      │
│  29%, fueled by the increasing reliance on social media for marketing tactics. Studies show that brands         │
│  receive an average of $6.50 for every $1 spent on influencer marketing, showcasing superior ROI compared to    │
│  traditional marketing methods. As the landscape becomes increasingly competitive, companies are investing in   │
│  a diverse range of influencers, from micro to macro, to effectively tap into niche audiences.                  │
│                                                                                                                 │
│  ## Current Trends                                                                                              │
│  The following trends have emerged prominently in 2025:                                                         │
│  - **Increased Focus on Authenticity**: Brands are seeking partnerships with influencers who resonate with      │
│  their values and target audience, fostering trust and authenticity.                                            │
│  - **Rise of Short-Form Video Content**: Platforms like TikTok have solidified short-form videos as a           │
│  preferred format for engaging audiences, leading to innovative campaign strategies.                            │
│  - **Sustainability Claims**: The rise of eco-conscious consumers has propelled brands to collaborate with      │
│  influencers advocating for sustainability.                                                                     │
│  - **Diversity and Inclusion**: Campaigns now prominen

In [4]:
from IPython.display import Markdown

Markdown(result.raw)

# Influencer Marketing Report 2025

## Executive Summary
Influencer marketing in 2025 has evolved into a vital strategy for brands looking to enhance their reach and engagement across digital platforms. Recent statistics indicate that influencer marketing spending in the United States is projected to exceed $10 billion this year, demonstrating significant growth from previous years. As consumers increasingly trust influencers over traditional advertising channels, businesses are leveraging authentic connections fostered by these content creators. Emerging platforms like TikTok continue to shape strategies, highlighting the need for brands to adapt to new trends quickly. This report outlines the current market landscape, recent trends, successful case studies, and provides actionable recommendations for C-suite executives to harness the full potential of influencer marketing.

## Market Overview
The influencer marketing sector has seen substantial growth in recent years, resulting in a market value estimated to reach approximately $16.4 billion in 2025. The annual growth rate for the industry is approx 29%, fueled by the increasing reliance on social media for marketing tactics. Studies show that brands receive an average of $6.50 for every $1 spent on influencer marketing, showcasing superior ROI compared to traditional marketing methods. As the landscape becomes increasingly competitive, companies are investing in a diverse range of influencers, from micro to macro, to effectively tap into niche audiences.

## Current Trends 
The following trends have emerged prominently in 2025:
- **Increased Focus on Authenticity**: Brands are seeking partnerships with influencers who resonate with their values and target audience, fostering trust and authenticity.
- **Rise of Short-Form Video Content**: Platforms like TikTok have solidified short-form videos as a preferred format for engaging audiences, leading to innovative campaign strategies.
- **Sustainability Claims**: The rise of eco-conscious consumers has propelled brands to collaborate with influencers advocating for sustainability.
- **Diversity and Inclusion**: Campaigns now prominently feature diverse influencers to reflect broader societal values and attract a wider audience.
- **Paid Collaborations and UGC**: User Generated Content (UGC) remains pivotal, with brands leveraging authentic posts from everyday users alongside paid influencer partnerships.

Statistics such as an estimated 80% of marketers reporting effective influencer campaigns further illustrate the impact influencers have on brand visibility and engagement.

## Case Studies
1. **Glossier**: The beauty brand has thrived by prioritizing influencer marketing, utilizing micro-influencers who generate genuine engagement. Campaigns generated significant buzz around product launches, reflecting the effectiveness of peer recommendations.
   
2. **Gymshark**: This fitness apparel brand effectively used influencer collaborations, particularly on TikTok, resulting in a 200% increase in sales during their influencer-led campaigns, showcasing the power of engaging in communities.

3. **Fenty Beauty**: Launched by Rihanna, Fenty's inclusive approach attracted a diverse group of influencers from various backgrounds, emphasizing representation in beauty advertising and resulting in notable sales growth.

## Recommendations
- **Invest in Authentic Influencer Relationships**: Establish long-term partnerships with influencers whose followers align with the brand’s target demographic.
- **Adapt to Platform-Specific Trends**: Regularly assess the performance of different content formats across platforms like Instagram and TikTok, and adapt campaigns accordingly.
- **Leverage Data Analytics**: Use analytics to measure the performance of influencer campaigns efficiently and identify areas for improvement.
- **Prioritize Ethical Marketing**: Align with brands that promote sustainability and inclusivity, as these values are increasingly significant to consumers.

## Conclusion
The influencer marketing landscape in 2025 presents both challenges and opportunities for brands. By understanding current trends, investing strategically, and fostering authentic relationships with influencers, organizations can enhance brand engagement and ultimately drive sales. Executives must be adaptable and forward-thinking to navigate this evolving field successfully.

This report serves as a guide for C-suite executives looking to refine their influencer marketing strategies and maximize success in the creator-driven market of 2025.

### Conclusion

In the agent task outputs above, you can see where the agent called the planning before starting the task. For more information on Planning, please reference our [documentation](https://docs.crewai.com/en/concepts/planning).

The Planing feature is an alternative to the **Reasoning** feature in CrewAI. There's a good chance both perform similar or the reasoning will be better.
We recommend trying both to see what works best for you.

Happy building! 🎉